In [ ]:
# default_exp network

# network

> Operations on networkx MultiDiGraph objects, culminating in the `Network` class central to this package. 

In [ ]:
#hide
from nbdev import *
from nbdev.imports import *
from nbdev.export import *
from nbdev.sync import *
from nbdev.showdoc import *

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
#export
import networkx as nx
import numpy as np
import sidis
from sidis import get,give,sort,push,refresh,NOT,COPY,XOR
import typing
from typing import Optional, Tuple, Dict, Callable, Union, Mapping, Sequence, Iterable, Hashable, List, Any
from collections import namedtuple

C:\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [ ]:
#export
def ring(N : int = 3,
         left : bool = True,
         right : bool = False,
         loop : bool = False):
    '''
    Return `g`, a ring topology networkx graph with `N` nodes.
    Booleans `left`, `right`, `loop` determine the directed edges.
    '''

    g=nx.MultiDiGraph()

    e=[]

    if left:
        e+=[(i,(i-1)%N) for i in range(N)]
    if right:
        e+=[(i,(i+1)%N) for i in range(N)]
    if loop:
        e+=[(i,i) for i in range(N)]

    g.add_nodes_from([i for i in range(N)])
    g.add_edges_from(e)

    return g

In [ ]:
g=ring(N=3,left=True,right=True,loop=True)

In [ ]:
#export
def adj(g : nx.MultiDiGraph, named=False):
    '''
    Returns list of (node, predecessors, successors) in list format.
    If named, uses named tuple.
    '''
    if named:
        t = namedtuple('Adj',['Node','In','Out'])
        adj = [t(n, list(g.predecessors(n)), list(g.successors(n))) for n in g.nodes]
    else:
        adj = [[n, list(g.predecessors(n)), list(g.successors(n))] for n in g.nodes]
    return adj

In [ ]:
adj(g, named=True)

[Adj(Node=0, In=[1, 2, 0], Out=[2, 1, 0]),
 Adj(Node=1, In=[2, 0, 1], Out=[0, 2, 1]),
 Adj(Node=2, In=[0, 1, 2], Out=[1, 0, 2])]

In [ ]:
#exporti
def table(iterable : Iterable, header : Iterable[str]):
    '''
    Creates a simple ASCII table from an iterable and a header.
    Modified from
    https://stackoverflow.com/questions/5909873/how-can-i-pretty-print-ascii-tables-with-python
    '''
    max_len = [len(x) for x in header]
    for row in iterable:
        row = [row] if type(row) not in (list, tuple) else row
        for index, col in enumerate(row):
            if max_len[index] < len(str(col)):
                max_len[index] = len(str(col))

    output = '|' + ''.join([h + ' ' * (l - len(h)) + '|' for h, l in zip(header, max_len)]) + '\n'

    for row in iterable:
        row = [row] if type(row) not in (list, tuple) else row
        output += '|' + ''.join([str(c) + ' ' * (l - len(str(c))) + '|' for c, l in zip(row, max_len)]) + '\n'

    return output


In [ ]:
#export
def print_graph(g : nx.MultiDiGraph,
               string=False):
    '''
    Print the 'node', predecessors', and 'successors' for every node in graph `g`.
    The predecessors are the nodes flowing into a node, 
    and the successors are the nodes flowing out.
    
    Example use:
        g=ring(N=3,left=True,right=True,loop=True)
        print_graph(g)
    '''
    data = adj(g)
    for i in range(len(data)):
        data[i][1]=', '.join([str(i) for i in data[i][1]])
        data[i][2]=', '.join([str(i) for i in data[i][2]])

    header=['Node', 'Predecessors', 'Successors']
    
    if not string:
        print(table(data,header))
    else:
        return table(data,header)


In [ ]:
print_graph(g)

|Node|Predecessors|Successors|
|0   |1, 2, 0     |2, 1, 0   |
|1   |2, 0, 1     |0, 2, 1   |
|2   |0, 1, 2     |1, 0, 2   |



In [ ]:
#export
def parse_kwargs(**kwargs):
    '''
    Evaluate delayed function calls by assigning 
    attributes as (func, *arg) tuples.
    Parse keyword arguments with the convention that
    kwarg = tuple ( callable , *args) be returned as
    kwarg = callable ( *args). 
    Example: kwargs = {a : (np.random.random,1)}
    becomes  kwargs = {a : np.random.random(1)} 
    each time this func is called.
    '''
    newkwargs={k:v for k,v in kwargs.items()}
    for k,v in kwargs.items():
        if type(v) is tuple and callable(v[0]):
            if len(v)==1:
                newkwargs[k]=v[0]()
            else:
                newkwargs[k]=v[0](*v[1:])
    return newkwargs

In [ ]:
parse_kwargs(a = (np.random.random,1) ) #evaluate func

{'a': array([0.39487206])}

In [ ]:
parse_kwargs(a = (np.random.random,1) ) #call again; diff result

{'a': array([0.82915321])}

In [ ]:
parse_kwargs(b=1, a = np.random.random) #no arg

{'b': 1, 'a': <function RandomState.random>}

In [ ]:
parse_kwargs(b=1, a = (np.random.random,) ) #default arg

{'b': 1, 'a': 0.30381956973271806}

In [ ]:
#export
def give_nodes(g : nx.MultiDiGraph,
               data : Dict[Hashable,dict] = None,
               nodes : Iterable = None,
               **kwargs):
    '''
    Parse and apply any 'kwargs' to a set of 'nodes'.
    If given, 'data' is a dict-of-dicts keyed by node.
    The inner dict is given to the corresponding node.
    '''
    if nodes is None:
        nodes=g.nodes
    
    if kwargs:
        [sidis.give(g.nodes[n],**parse_kwargs(**kwargs)) for n in nodes]
    
    if data:
        for k,v in data.items():
            try:
                g.nodes[k].update(parse_kwargs(**v))
            except KeyError:
                pass


In [ ]:
give_nodes(g,a=(np.random.random,1))
g.nodes(data=True)

NodeDataView({0: {'a': array([0.3092636])}, 1: {'a': array([0.38798102])}, 2: {'a': array([0.40887628])}})

In [ ]:
give_nodes(g,{0:dict(b=1)})
g.nodes(data=True)

NodeDataView({0: {'a': array([0.3092636]), 'b': 1}, 1: {'a': array([0.38798102])}, 2: {'a': array([0.40887628])}})

In [ ]:
give_nodes(g,nodes=[2],c=2)
g.nodes(data=True)

NodeDataView({0: {'a': array([0.3092636]), 'b': 1}, 1: {'a': array([0.38798102])}, 2: {'a': array([0.40887628]), 'c': 2}})

In [ ]:
#export
def parse_edges(edges : Union[tuple,List[tuple]], 
                default_key : Hashable = 0
               ):
    '''
    Parse a single edge or list of edges
    into a list of 3-tuples for iterating over 
    a MultiDiGraph, which requires keys. 
    '''
    if type(edges) is tuple:
        edges=[edges]
    if type(edges) is not list:
        edges=list(edges)
    for i in range(len(edges)):
        if len(edges[i])==4: #discard data, last entry
            edges[i]=(edges[i][0],edges[i][1],edges[i][2])
        if len(edges[i])==2: #include key, 3rd entry
            edges[i]=(edges[i][0],edges[i][1],default_key)
    return edges

In [ ]:
parse_edges( (0,0) )

[(0, 0, 0)]

In [ ]:
#export
def give_edges(g : nx.MultiDiGraph,
               data : Dict[Hashable,dict] = None,
               edges : Iterable = None,
               **kwargs):
    '''
    Parse and apply any 'kwargs' to a set of 'edges'.
    If given, 'data' is a dict-of-dicts keyed by edge.
    The inner dict is given to the corresponding edge.
    '''
    if edges is None:
        edges=g.edges
        
    edges = parse_edges(edges)
    
    if kwargs:
        [sidis.give(g.edges[e],**parse_kwargs(**kwargs)) for e in edges]
    
    if data:
        for k,v in data.items():
            try:
                g.edges[k].update(parse_kwargs(**v))
            except KeyError:
                pass

In [ ]:
give_edges(g,d=1)
g.edges(data=True)

OutMultiEdgeDataView([(0, 2, {'d': 1}), (0, 1, {'d': 1}), (0, 0, {'d': 1}), (1, 0, {'d': 1}), (1, 2, {'d': 1}), (1, 1, {'d': 1}), (2, 1, {'d': 1}), (2, 0, {'d': 1}), (2, 2, {'d': 1})])

In [ ]:
#export
def node_attrs(g):
    '''
    Unique node data keys.
    '''
    return list(set(sidis.flatten([list(t[-1].keys())
                        for t in list(g.nodes(data=True))])))

In [ ]:
node_attrs(g)

['b', 'a', 'c']

In [ ]:
#export
def edge_attrs(g):
    '''
    Unique edge data keys.
    '''
    return list(set(sidis.flatten([list(t[-1].keys())
                        for t in list(g.edges(data=True))])))

In [ ]:
edge_attrs(g)

['d']

In [ ]:
#export
def node_data(g,*args):
    '''
    Return node attributes 'args' as an array.
    NOTE: The ordering of the array corresponds to the
    ordering of the nodes in the graph.
    '''
    if not args:
        args=node_attrs(g)

    node_data={}

    [sidis.give(node_data,str(arg), np.squeeze(np.array([sidis.get(g.nodes[n],arg) for n in g.nodes])))
        for arg in args]

    return node_data

In [ ]:
node_data(g)

{'b': array([1, None, None], dtype=object),
 'a': array([0.3092636 , 0.38798102, 0.40887628]),
 'c': array([None, None, 2], dtype=object)}

In [ ]:
#export
def edge_data(g,*args):
    '''
    Return edge attributes 'args' as an array.
    NOTE: The ordering of the array corresponds to the
    ordering of the edges in the graph.
    '''
    if not args:
        args=edge_attrs(g)

    edge_data={}

    [sidis.give(edge_data,str(arg), np.array([sidis.get(g.edges[e],arg) for e in g.edges]))
        for arg in args]

    return edge_data

In [ ]:
edge_data(g)

{'d': array([1, 1, 1, 1, 1, 1, 1, 1, 1])}

In [ ]:
#export
def argwhere(*args : List[np.ndarray]):
    '''
    Simplified version of np.argwhere for multiple arrays.
    Returns list of indices where args hold.
    '''
    if not args:
        return None
    elif len(args)==1:
        return list(np.ravel(np.argwhere(args[0])).astype(int))
    else:
        i=[] #indices
        for arg in args:
            res=list(np.ravel(np.argwhere(arg)).astype(int))
            i+=[res]
        if len(i)==1:
            i=i[0]
        if np.any(i):
            return list(i)

In [ ]:
a=np.array([0,1,2])
argwhere(a==0,a==1,a>1,a==10)

C:\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:87: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


[[0], [1], [2], []]

In [ ]:
#export
def kwargwhere(g : nx.MultiDiGraph,**kwargs : Dict[str,Any]):
    '''
    Return the node and edges where
    the kwarg equalities hold in the graph.
    '''
    node_k=node_attrs(g)
    edge_k=edge_attrs(g)
    node_i=[]
    edge_i=[]
    for k,v in kwargs.items():
        n_i=[]
        e_i=[]
        if k in node_k:
            for n in g.nodes:
                if g.nodes[n].get(k)==v:
                    n_i+=[n]
            node_i+=[n_i]
        if k in edge_k:
            for e in g.edges:
                if g.edges[e].get(k)==v:
                    e_i+=[e]
            edge_i+=[e_i]

    if len(node_i)==1:
        node_i=node_i[0]
    if len(edge_i)==1:
        edge_i=edge_i[0]
    if node_i and edge_i:
        return node_i,edge_i
    elif node_i:
        return node_i
    elif edge_i:
        return edge_i

In [ ]:
print(g.nodes(data=True))

[(0, {'a': array([0.3092636]), 'b': 1}), (1, {'a': array([0.38798102])}), (2, {'a': array([0.40887628]), 'c': 2})]


In [ ]:
kwargwhere(g,b=1)

[0]

In [ ]:
kwargwhere(g,d=1)

[(0, 2, 0),
 (0, 1, 0),
 (0, 0, 0),
 (1, 0, 0),
 (1, 2, 0),
 (1, 1, 0),
 (2, 1, 0),
 (2, 0, 0),
 (2, 2, 0)]

In [ ]:
g.edges[(0,0,0)]['b']=1
print(g.edges(data=True))

[(0, 2, {'d': 1}), (0, 1, {'d': 1}), (0, 0, {'d': 1, 'b': 1}), (1, 0, {'d': 1}), (1, 2, {'d': 1}), (1, 1, {'d': 1}), (2, 1, {'d': 1}), (2, 0, {'d': 1}), (2, 2, {'d': 1})]


In [ ]:
kwargwhere(g,b=1)

([0], [(0, 0, 0)])

In [ ]:
#export
def where(g,*args,**kwargs):
    '''
    Combine the 'argwhere' and 'kwargwhere' functions for the graph.
    '''
    arg_i=argwhere(*args)
    kwarg_i=kwargwhere(g,**kwargs)
    if arg_i and kwarg_i:
        return arg_i,kwarg_i
    elif arg_i:
        return arg_i
    elif kwarg_i:
        return kwarg_i

In [ ]:
where(g,node_data(g)['a']>0.5,b=1)

([0], [(0, 0, 0)])

In [ ]:
#export
def parse_lengths(g : nx.MultiDiGraph,
                  edges : Union[tuple,List[tuple]],
                  lengths : Union[str,int,List[int]] = 1) -> Union[list,List[list]]:
    '''
    Convert `lengths` corresponding to attributes of each edge into a list of lists.
    `lengths` can be a single integer, an integer for each edge, or a string
    giving the edge attribute holding the length.
    '''
    if type(lengths) is int:
        lengths=[lengths for i in range(len(edges))]
    elif type(lengths) is str:
        lengths=[g.edges[e][lengths] for e in edges]
    return lengths

In [ ]:
#export
def edge_to_line(g : nx.MultiDiGraph,
                   edges : Union[tuple,List[tuple]],
                   lengths : Union[str,int,List[int]] = 1,
                   labels : callable = lambda g,edge,length,accum : len(list(g.nodes))+accum
                  ) -> Union[list,List[list]]:
    '''
    Convert an edge or list of edges with a line path (or list of line paths),
    which directly connect the previous endpoints of the edge through new nodes.
    Every edge is replaced with 'length' number of new nodes,
    which may be an integer or list of integers corresponding to the
    edge or list of edges. These new nodes take the name from `labels`,
    which default to the number of nodes in the graph after each addition.
    Returns the paths as lists of node labels.
    '''
    #parse args
    edges=parse_edges(edges)
    lengths=parse_lengths(g,edges,lengths)
    #generate line paths
    lines=[[] for l in lengths]
    accum=0
    for i,edge in enumerate(edges):
        for length in range(lengths[i]):
            accum+=length #new node within path
            lines[i]+=[labels(g=g,edge=edge,length=length,accum=accum)]
        accum+=1 #increment with new node starting new edge path
    return [[e[0]]+lines[i]+[e[1]] for i,e in enumerate(edges)]



In [ ]:
g=ring(left=True,right=True)
edges=g.out_edges(0)
print(edges)
edge_to_line(g,edges,lengths=2)

[(0, 2), (0, 1)]


[[0, 3, 4, 2], [0, 5, 6, 1]]

In [ ]:
edge_to_line(g,edges,lengths=2,labels=lambda g,edge,length,accum: (edge[0],edge[1],length))

[[0, (0, 2, 0), (0, 2, 1), 2], [0, (0, 1, 0), (0, 1, 1), 1]]

In [ ]:
g=ring(left=True)
edge_to_line(g,g.edges,lengths=[0,1,2])

[[0, 2], [1, 4, 0], [2, 5, 6, 1]]

In [ ]:
#export
def edge_to_tree(g,
                 edges : Union[tuple,List[tuple]],
                 lengths : Union[str,int,List[int]] = 1,
                 labels : Dict[str,callable] = {
                    'trunk' : lambda g,root,leaves,accum : len(list(g.nodes))+accum,
                    'branch' : lambda g,trunk,root,leaf,length : trunk+length
                 }
                ) -> Union[list,List[list]]:
    '''
    Convert an edge or list of edges with a tree path (or list of tree paths),
    which connect the previous endpoints of the edge through new nodes.
    The new nodes have a tree structure, consolidating shared base nodes.
    Every edge is replaced with a path of 'length' number of nodes,
    which may be an integer or list of integers corresponding to the
    edge or list of edges. These nodes take the name from `labels`,
    which default to the number of nodes in the graph after each addition.
    Returns the paths as lists of node labels.
    '''
    #parse args
    trunk_label,branch_label = labels['trunk'],labels['branch']
    edges=parse_edges(edges)
    lengths=parse_lengths(g,edges,lengths)
    #get roots (first node in edges) and leaves (2nd node in edge)
    roots = [e[0] for e in edges]
    leaves = [e[1] for e in edges]
    #get outputs of each root, for optional labeling purposes only
    leaves_with_root = {r:[e[1] for e in edges if e[0]==r] for r in roots}
    #get max path length associated with each root, used to grow tree trunk that branch paths split off from
    max_lengths = {r:max([lengths[i] for i,e in enumerate(edges) if e[0]==r]) for r in roots}
    accum=0
    trunks={}
    #grow trunks
    for i,r in enumerate(set(roots)):
        trunks[r]=trunk_label(g=g,root=r,leaves=leaves_with_root[r],accum=accum)
        accum+=max_lengths[r]
    trunks = [trunks[r] for r in roots]
    #give branch paths
    branches = [[branch_label(g=g,trunk=trunks[i],length=l,root=r,leaf=leaves[i]) for l in range(1,lengths[i])] 
                for i,r in enumerate(roots)]
    #grow trees (list of paths)
    trees = [[root]+[trunk]+branch+[leaf] for root,trunk,branch,leaf in zip(roots,trunks,branches,leaves)]
    return trees

In [ ]:
g=ring(left=True,right=True)
edge_to_tree(g,g.out_edges(0),lengths=2) #compare to line, which produces new node in 2nd path

[[0, 3, 4, 2], [0, 3, 4, 1]]

In [ ]:
edge_to_tree(g,g.out_edges(0),
               lengths=2,
               labels=dict(
               trunk=lambda g,root,leaves,accum : leaves,
               branch=lambda g,trunk,root,leaf,length : trunk+[length]
               )
              )

[[0, [2, 1], [2, 1, 1], 2], [0, [2, 1], [2, 1, 1], 1]]

In [ ]:
for i,n in enumerate(g.nodes):
    for e in g.out_edges(n,keys=True):
        g.edges[e]['weight']=i
print(g.edges(data=True))

[(0, 2, {'weight': 0}), (0, 1, {'weight': 0}), (1, 0, {'weight': 1}), (1, 2, {'weight': 1}), (2, 1, {'weight': 2}), (2, 0, {'weight': 2})]


In [ ]:
edge_to_tree(g,g.edges,lengths='weight')

[[0, 3, 2], [0, 3, 1], [1, 3, 0], [1, 3, 2], [2, 4, 5, 1], [2, 4, 5, 0]]

In [ ]:
#export
def convert_edges_to_paths(g : nx.MultiDiGraph,
                 edges : Union[tuple,List[tuple]],
                 paths : Union[list,List[list],callable] = edge_to_tree,
                 lengths : Union[int,List[int]] = 1,
                 labels : Union[None,callable,Dict[str,callable]] = None,
                 node_attrs : Union[dict,List[dict]] = {},
                 return_paths = False,
                 **edge_attrs
                ) -> None:
    '''
    Modify a graph in place by converting `edges` to `paths` of new nodes.
    `paths` may be a list of paths or a callable producing those paths.
    If a callable, the arguments must be `edges`, `lengths`, and `labels`.
    See `edge_to_line` and `edge_to_tree` for descriptions on callables.
    The new nodes inherit `node_attrs`, which may be a list, and every new 
    edge inherits `edge_attrs`, which applies to all edges. 
    '''
    #parse attributes
    edges=parse_edges(edges)
    
    if callable(paths):
        if labels is None:
            paths = paths(g,edges=edges,lengths=lengths)
        else:
            paths = paths(g,edges=edges,lengths=lengths,labels=labels)
            
    if sidis.depth(paths)!=2: #ensure it's iterable!
        paths=[paths]
        
    for i in range(len(paths)): #ensure endpoints are endpoints of edges
        start,stop=edges[i][0],edges[i][1]
        if paths[i][0]!=start:
            paths[i]=[start]+paths[i]
        if paths[i][-1]!=stop:
            paths[i]=paths[i]+[stop]
        
    if type(node_attrs) is not list: #ensure attrs match each node in path
        node_attrs=[[parse_kwargs(**node_attrs) for i in range(len(path))] for path in paths]
    
    #modification algorithm
    for i,edge in enumerate(list(edges)):
        path=paths[i]

        new_nodes=path[1:-1]

        nx.add_path(g,path,**parse_kwargs(**edge_attrs))

        g.remove_edge(*edge)
        for j,n in enumerate(new_nodes):
   
            give_nodes(g,nodes=[n],data=node_attrs[i][j],**node_attrs[i][j]) #attrs keyed by node, or k,v pairs
    
    if return_paths:
        return paths



In [ ]:
g=ring(left=True,right=True)
print_graph(g)
convert_edges_to_paths(g,edges=g.edges,paths=edge_to_tree,lengths=1)
print_graph(g)

|Node|Predecessors|Successors|
|0   |1, 2        |2, 1      |
|1   |2, 0        |0, 2      |
|2   |0, 1        |1, 0      |

|Node|Predecessors|Successors|
|0   |4, 5        |3         |
|1   |3, 5        |4         |
|2   |3, 4        |5         |
|3   |0           |2, 1      |
|4   |1           |0, 2      |
|5   |2           |1, 0      |



In [ ]:
g=ring(left=True)
print_graph(g)
edges=list(g.out_edges(0))
convert_edges_to_paths(g,edges=edges,paths=['whatever','we','want'])
print_graph(g)

|Node|Predecessors|Successors|
|0   |1           |2         |
|1   |2           |0         |
|2   |0           |1         |

|Node    |Predecessors|Successors|
|0       |1           |whatever  |
|1       |2           |0         |
|2       |want        |1         |
|whatever|0           |we        |
|we      |whatever    |want      |
|want    |we          |2         |



In [ ]:
#export
def convert_paths_to_edges(g : nx.MultiDiGraph,
                   paths : Union[list,List[list]],
                   return_edges = False,
                   **edge_attrs) -> None:
    '''
    Modify a graph in place by converting `paths` to edges between the 
    endpoints of those paths. Every new edge inherits `edge_attrs`.
    Acts as the invertse of `convert_edges_to_paths`.
    '''
    #parse args
    if sidis.depth(paths)==1:
        paths=[paths]
    
    #modify in place
    all_edges = []
    for path in paths:
        left=path[0]
        right=path[-1]
        #see networkx docs for this, produces list of edges connecting nodes in chain
        edges=list(nx.utils.pairwise(nx.utils.chain((left,), path)))
        all_edges+=[edges]
        g.remove_edges_from(edges)
        links=path[1:-1]
        g.add_edge(left,right,**edge_attrs)
        for node in links:
            if not g.in_edges(node) and not g.out_edges(node):
                g.remove_node(node)
    if return_edges:
        return all_edges

In [ ]:
g=ring(left=True)
print('graph before conversion')
print_graph(g)
edges=list(g.out_edges(0))
paths=edge_to_tree(g,edges)
print('edges to convert', edges)
print('paths from edges', paths,'\n')
convert_edges_to_paths(g,edges,paths)
print('graph after converting edges to paths')
print_graph(g)
print('graph after converting paths to edges, same as original')
convert_paths_to_edges(g,paths)
print_graph(g)

graph before conversion
|Node|Predecessors|Successors|
|0   |1           |2         |
|1   |2           |0         |
|2   |0           |1         |

edges to convert [(0, 2, 0)]
paths from edges [[0, 3, 2]] 

graph after converting edges to paths
|Node|Predecessors|Successors|
|0   |1           |3         |
|1   |2           |0         |
|2   |3           |1         |
|3   |0           |2         |

graph after converting paths to edges, same as original
|Node|Predecessors|Successors|
|0   |1           |2         |
|1   |2           |0         |
|2   |0           |1         |



In [ ]:
#export
def add_star(g : nx.MultiDiGraph,
             nodes : List[Hashable],
             view='in',
             **edge_attrs):
    '''
    Extends `nx.add_star` to include direction from `view`.
    `add_star` connects the first node in the list of `nodes`
    to all others, with the direction indicated by `view`.
    '''
    assert view=='in' or view=='out' or view=='both', \
        'Require view = "in" or view = "out" or view = "both"'
    nlist = iter(nodes)
    try:
        v = next(nlist)
    except StopIteration:
        return
    g.add_node(v)
    if view is 'out':
        edges = ((v, n) for n in nlist)
    elif view is 'in':
        edges = ((n, v) for n in nlist)
    elif view is 'both':
        nlist=list(nlist)
        edges = [(v, n) for n in nlist]+[(n, v) for n in nlist]
    g.add_edges_from(edges, **edge_attrs)

In [ ]:
g=ring()
print('in')
add_star(g,[3,0,1,2],view='in')
print_graph(g)
g=ring()
add_star(g,[3,0,1,2],view='out')
print('out')
print_graph(g)
g=ring()
add_star(g,[3,0,1,2],view='both')
print('both')
print_graph(g)

in
|Node|Predecessors|Successors|
|0   |1           |2, 3      |
|1   |2           |0, 3      |
|2   |0           |1, 3      |
|3   |0, 1, 2     |          |

out
|Node|Predecessors|Successors|
|0   |1, 3        |2         |
|1   |2, 3        |0         |
|2   |0, 3        |1         |
|3   |            |0, 1, 2   |

both
|Node|Predecessors|Successors|
|0   |1, 3        |2, 3      |
|1   |2, 3        |0, 3      |
|2   |0, 3        |1, 3      |
|3   |0, 1, 2     |0, 1, 2   |



In [ ]:
#export
def relabel_graph(g : nx.MultiDiGraph,
            mapping : Union[None,callable,dict] = None):
    '''
    Relabel nodes in place with desired 'mapping', and store the 
    `mapping` and `inverse_mapping` as attributes of `g`. 
    Can be called again without args to relabel to the original map,
    which switches the `mapping` and `inverse_mapping`.
    If `mapping` is None and `g` has no `mapping`, 
    defaults to replacing nodes with integers.
    If `mapping` is None and `g` has a `mapping`, uses that.
    Otherwise, `mapping` is a callable or dict keyed with old node labels 
    as keys and new node labels as values.
    '''
    if mapping is None:
        if not g.__dict__.get('mapping'):
            mapping={n:i for i,n in enumerate(g.nodes)}
        else:
            mapping=g.mapping
            
    elif callable(mapping):
        mapping=mapping(g)
    
    inverse_mapping={v:k for k,v in mapping.items()}
    
    gnew=nx.relabel_nodes(g,mapping,copy=True)
    g.__dict__.update(gnew.__dict__)
    
    g.mapping=inverse_mapping
    g.inverse_mapping=mapping

In [ ]:
g=ring()
print(g.nodes)

[0, 1, 2]


In [ ]:
relabel_graph(g,mapping={n:4+n for n in g.nodes})
print(g.nodes)

[4, 5, 6]


In [ ]:
print(g.mapping)
print(g.inverse_mapping)

{4: 0, 5: 1, 6: 2}
{0: 4, 1: 5, 2: 6}


In [ ]:
relabel_graph(g)
print(g.nodes)

[0, 1, 2]


In [ ]:
relabel_graph(g)
print(g.nodes)

[4, 5, 6]


In [ ]:
relabel_graph(g,mapping=lambda g:{n:-n for n in g.nodes})
print(g.nodes)

[-4, -5, -6]


In [ ]:
relabel_graph(g)
print(g.nodes)

[4, 5, 6]


In [ ]:
g=ring(left=True,right=False,loop=False)

In [ ]:
#export
def sort_graph(g : nx.MultiDiGraph,
               nodes_by='in_degree', #g.in_degree, #sorting this function over nodes
               node_key=lambda t:sidis.get(t,-1,-1), #last element of sorting tuple
               node_args=(), #not accessing any attributes by default
               nodes_ascending=True,
               edges_by=None, #not generating function evals to sort
               edge_key=None,#orders edges, defaults to linear comb of node sort
               edge_args=(), #not accessing any edge attrs by default
               edges_ascending=False,
               relabel=False #relabel to integers
              ) -> None:
    '''
    Sort the graph in place by changing node and edge order.
    See `sidis.sort` documentation for explanation of by, key, and args.
    Default behavior is to sort nodes by in-degree, and edges by increasing node label,
    after relabling nodes to integers. Stores result in 'sorting' attribute.
    '''
    #parse args; get node sorting attr if str
    if type(nodes_by) is str:
        nodes_by=get(g,nodes_by)
    #if no edge key given default to ordering by linear comb of node func
    if edge_key is None:
        edge_key=lambda t:100*nodes_by(t[0])-10*nodes_by(t[1])
    
    #sort nodes
    node_sorting=sidis.sort(g.nodes,
                            *node_args,
                            by=nodes_by,
                            key=node_key,
                            reverse=nodes_ascending)

    #sort returns tuples of (node,nodes_by(node)), so extract nodes and data
    if nodes_by is None:
        nodes=[(n,g.nodes[n]) for n in node_sorting]
    else:
        nodes=[(n[0],g.nodes[n[0]]) for n in node_sorting]  
    
    #sort edges
    edge_sorting=sidis.sort(list(g.edges(keys=True)),
                            *edge_args,
                            by=edges_by,
                            key=edge_key,
                            reverse=edges_ascending)
    
    #extract edge,data tuple
    if edges_by is None:
        edges=[(*e,g.edges[e]) for e in edge_sorting]
    else:
        edges=[(*e[0],g.edges[e[0]]) for e in edge_sorting]        
    
    #wipe graph and add new nodes/edges in order
    g.clear()
    g.add_nodes_from(nodes)
    g.add_edges_from(edges)
    
    #relabel to new ranking if desired
    if relabel:
        relabel_graph(g)
        new_node_sorting=[]
        for node,rank in node_sorting:
            new_node_sorting+=[(g.inverse_mapping[node],rank)]
        node_sorting=new_node_sorting
    
    sorting=nx.utils.groups(dict(node_sorting))
    g.sorting={k:list(v) for k,v in sorting.items()}             

In [ ]:
g=ring(left=True,right=False,loop=False)
g.add_edges_from([(2,2),(3,2),(1,1)])
G=g.copy()
print('nodes before sorting', g.nodes)
print('edges before sorting', g.edges(keys=False))
print('\ngraph before sorting')
print_graph(g)
sort_graph(g,relabel=False)

print('nodes after sorting', g.nodes)
print('edges after sorting', g.edges(keys=False))
print('\ngraph after sorting')
print_graph(g)

sort_graph(G,relabel=True)
print('nodes after sorting and relabeling', G.nodes)
print('edges after sorting and relabeling', G.edges(keys=False))
print('\ngraph after sorting and relabeling')
print_graph(G)

nodes before sorting [0, 1, 2, 3]
edges before sorting [(0, 2), (1, 0), (1, 1), (2, 1), (2, 2), (3, 2)]

graph before sorting
|Node|Predecessors|Successors|
|0   |1           |2         |
|1   |2, 1        |0, 1      |
|2   |0, 2, 3     |1, 2      |
|3   |            |2         |

nodes after sorting [2, 1, 0, 3]
edges after sorting [(2, 2), (2, 1), (1, 1), (1, 0), (0, 2), (3, 2)]

graph after sorting
|Node|Predecessors|Successors|
|2   |3, 0, 2     |2, 1      |
|1   |1, 2        |1, 0      |
|0   |1           |2         |
|3   |            |2         |

nodes after sorting and relabeling [0, 1, 2, 3]
edges after sorting and relabeling [(0, 0), (0, 1), (1, 1), (1, 2), (2, 0), (3, 0)]

graph after sorting and relabeling
|Node|Predecessors|Successors|
|0   |0, 2, 3     |0, 1      |
|1   |0, 1        |1, 2      |
|2   |1           |0         |
|3   |            |0         |



In [ ]:
g=nx.MultiDiGraph()
g.add_edges_from([('a','c'),('b','a'),('b','b'),('c','b'),('c','c'),('d','c')])
G=g.copy()
print('nodes before sorting', g.nodes)
print('edges before sorting', g.edges(keys=False))
print('\ngraph before sorting')
print_graph(g)
sort_graph(g,relabel=False)
print('nodes after sorting', g.nodes)
print('edges after sorting', g.edges(keys=False))
print('\ngraph after sorting')
print_graph(g)
sort_graph(G,relabel=True)
print('nodes after sorting and relabeling', G.nodes)
print('edges after sorting and relabeling', G.edges(keys=False))
print('\ngraph after sorting and relabeling')
print_graph(G)

nodes before sorting ['a', 'c', 'b', 'd']
edges before sorting [('a', 'c'), ('c', 'b'), ('c', 'c'), ('b', 'a'), ('b', 'b'), ('d', 'c')]

graph before sorting
|Node|Predecessors|Successors|
|a   |b           |c         |
|c   |a, c, d     |b, c      |
|b   |b, c        |a, b      |
|d   |            |c         |

nodes after sorting ['c', 'b', 'a', 'd']
edges after sorting [('c', 'c'), ('c', 'b'), ('b', 'b'), ('b', 'a'), ('a', 'c'), ('d', 'c')]

graph after sorting
|Node|Predecessors|Successors|
|c   |d, a, c     |c, b      |
|b   |b, c        |b, a      |
|a   |b           |c         |
|d   |            |c         |

nodes after sorting and relabeling [0, 1, 2, 3]
edges after sorting and relabeling [(0, 0), (0, 1), (1, 1), (1, 2), (2, 0), (3, 0)]

graph after sorting and relabeling
|Node|Predecessors|Successors|
|0   |0, 2, 3     |0, 1      |
|1   |0, 1        |1, 2      |
|2   |1           |0         |
|3   |            |0         |



In [ ]:
G.sorting

{3: [0], 2: [1], 1: [2], 0: [3]}

In [ ]:
g.sorting

{3: ['c'], 2: ['b'], 1: ['a'], 0: ['d']}

In [ ]:
#export
class Network(nx.MultiDiGraph):
    '''
    MultiDiGraph with sorting and labeling methods,
    and easy manipulation of node/edge properties.
    '''
    def __init__(self,
                 g=None,
                 view='out',
                 relabel=False,
                 **sort_kwargs
                 ):
    
        super().__init__(g)
        
        self.view(view)
        
        if g is not None:
            self.sort(relabel=relabel,**sort_kwargs)
            if node_attrs(g):
                self.node_data(save=True)
            if edge_attrs(g):
                self.edge_data(save=True)
            
    
    def view(self,view='in'):
        '''
        Change default edge view
        '''
        if view=='in':
            self.edge_view=nx.classes.reportviews.InMultiEdgeView

        elif view=='out':
            self.edge_view=nx.classes.reportviews.OutMultiEdgeView
            
    
    @nx.MultiDiGraph.edges.getter
    def edges(self):
        return self.edge_view(self)
    
    
    def sort(self,**kwargs):
        sort_graph(self,**kwargs)
        

    def relabel(self,mapping=None):
        relabel_graph(self,mapping)
        

    def node_attrs(self):
        return node_attrs(self)
    

    def edge_attrs(self):
        return edge_attrs(self)
    
    
    def attrs(self):
        return self.node_attrs()+self.edge_attrs()
    

    def give_nodes(self,data=None,nodes=None,**kwargs):
        give_nodes(self,data=data,nodes=None,**kwargs)
            

    def give_edges(self,data=None,edges=None,**kwargs):
        give_edges(self,data=data,edges=edges,**kwargs)
            
    
    def give_self(self,data=None,**kwargs):
        '''
        Give any arg dict and kwargs to self as attrs
        '''
        try:
            give(self,**parse_kwargs(**kwargs))
        except:
            pass
        
        if data:
            for k,v in data.items():
                try:
                    self.__dict__.update(parse_kwargs(**v))
                except KeyError:
                    pass
            
            
    def clear_nodes(self,*args):
        '''
        Remove arg entries from node dict
        '''
        if not args:
            args=self.node_attrs()
        [[give(self.node[n],arg,None) for n in self.nodes] for arg in args]
        
    
    def clear_edges(self,*args):
        '''
        Remove arg entries from edge dict
        '''
        if not args:
            args=self.edge_attrs()
            
        [[give(self.edges[e],arg,None) for e in self.edges] for arg in args]
        

    def clear_self(self,*args):
        if not args:
            args=list(self.node_attrs())+list(self.edge_attrs())
        [give(self,arg,None) for arg in args]
        

    def node_data(self,*args,save=False):
        '''
        Set node attributes as self attribute array. 
        '''
        if not save:
            return node_data(self,*args)
        else:
            self.give_self(**node_data(self,*args))


    def edge_data(self,*args,save=False):        
        if not save:
            return edge_data(self,*args)
        else:
            self.give_self(**edge_data(self,*args))
            
    
    def self_data(self,*args):
        if not args:
            args=self.self_attrs()
        return {k:get(self,k) for k in args}
    

    def swap_data(self):
        '''
        Switch the data location of node/edge/self attributes.
        '''
        edges=list(self.edges)
        for attr in self.edge_attrs():
            if not np.any(get(self,attr)): #if not saved in self
                self.edge_data(attr,save=True) #save from edge
                self.clear_edges(attr) #and clear from edge
            else: #saved in self
                for i,e in enumerate(edges): #give to edge
                    self.give_edges(edges=[e],**{attr:get(self,attr,i)})
                self.clear_self(attr) #clear from self when done

        nodes=list(self.nodes) #repeat for nodes
        for attr in self.node_attrs():
            if not np.any(get(self,attr)):
                self.node_data(attr,save=True)
                self.clear_nodes(attr)
            else:
                for i,n in enumerate(nodes):
                    self.give_nodes(nodes=[n],**{attr:get(self,attr,i)})
                self.clear_self(attr)
                
                    
    def convert_edges(self,
            edges : Union[tuple,List[tuple]] = None,
            paths : Union[list,List[list],callable] = edge_to_tree,
            lengths : Union[int,List[int]] = 1,
            labels : Union[None,callable,Dict[str,callable]] = None,
            node_attrs : Union[dict,List[dict]] = {},
            relabel = False,
            **edge_attrs):
        
        if edges is None:
            edges=list(self.edges)
            
        self.return_paths=convert_edges_to_paths(self,edges=edges,paths=paths,lengths=lengths,labels=labels,
                               return_paths=True,**edge_attrs)
        
        self.sort(relabel=relabel)
        
        
    def convert_paths(self,paths,relabel=False,**edge_attrs):
        self.return_edges=convert_paths_to_edges(self,paths,return_edges=True,**edge_attrs)
        self.sort(relabel=relabel)
        
    def unconvert_edges(self,relabel=False):
        self.convert_paths(paths=self.return_paths)
        self.sort(relabel=relabel)
    
    def unconvert_paths(self,relabel=False):
        self.convert_edges(edges=self.return_edges)
        self.sort(relabel=relabel)
        
    def where(self,*args,**kwargs):
        return where(self,*args,**kwargs)
    
    def edgewhere(self,*args):
        return np.array(self.edges)[self.where(*args)]
    
    def nodewhere(self,*args):
        return np.array(self.nodes)[self.where(*args)]
    
    def __str__(self):
        return print_graph(self,string=True)
    
    def __repr__(self):
        s=str(self)
        S=super().__repr__()
        spl=s.split('\n')
        if len(spl)>10:
            spl=spl[:5]+['...']+spl[-5:]
        return S+'\n'+('\n').join(spl)

In [ ]:
g=ring(left=True,right=False,loop=False)
g.add_edges_from([(2,2),(3,2),(1,1)])

for i,n in enumerate(g.nodes):
    g.nodes[n]['n']=i
    
for i,e in enumerate(g.edges):
    g.edges[e]['e']=i
    
G=Network(g,relabel=True)

In [ ]:
G

|Node|Predecessors|Successors|
|0   |0, 2, 3     |0, 1      |
|1   |0, 1        |1, 2      |
|2   |1           |0         |
|3   |            |0         |

In [ ]:
G.n

array([2, 1, 0, 3])

In [ ]:
G.e

array([4, 3, 2, 1, 0, 5])

In [ ]:
G.nodes(data=True)

NodeDataView({0: {'n': 2}, 1: {'n': 1}, 2: {'n': 0}, 3: {'n': 3}})

In [ ]:
G.edges(data=True)

OutMultiEdgeDataView([(0, 0, {'e': 4}), (0, 1, {'e': 3}), (1, 1, {'e': 2}), (1, 2, {'e': 1}), (2, 0, {'e': 0}), (3, 0, {'e': 5})])

In [ ]:
G.where(G.n==0,e=5)

([2], [(3, 0, 0)])

In [ ]:
G.edgewhere(G.e>=4)

array([[0, 0, 0],
       [3, 0, 0]])

In [ ]:
G.nodewhere(G.n>=4) #bug

array([[0, 1, 2, 3]])

In [ ]:
G.convert_edges(edges=G.edgewhere(G.e>3),lengths=1)

In [ ]:
G

|Node|Predecessors|Successors|
|0   |2, 4, 5     |1, 4      |
|1   |1, 0        |1, 2      |
|2   |1           |0         |
|4   |0           |0         |
|5   |3           |0         |
|3   |            |5         |

In [ ]:
G.unconvert_edges()
G

|Node|Predecessors|Successors|
|0   |3, 2, 0     |0, 1      |
|1   |1, 0        |1, 2      |
|2   |1           |0         |
|3   |            |0         |

In [ ]:
G.view('in')
G.edges

InMultiEdgeView([(3, 0, 0), (2, 0, 0), (0, 0, 0), (1, 1, 0), (0, 1, 0), (1, 2, 0)])

In [ ]:
G.view('out')
G.edges

OutMultiEdgeView([(0, 0, 0), (0, 1, 0), (1, 1, 0), (1, 2, 0), (2, 0, 0), (3, 0, 0)])

In [ ]:
refresh()